In [8]:
pip install lightgbm

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.3 MB 2.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.6/1.3 MB 6.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.1/1.3 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 8.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
df = pd.read_csv(r"H:\Sparsh Ehsaas Documents\Ehsaas\ML\data.csv\data.csv")
print(df.shape)
print(df.head())

(169909, 19)
   acousticness                                   artists  danceability  \
0         0.995                       ['Carl Woitschach']         0.708   
1         0.994  ['Robert Schumann', 'Vladimir Horowitz']         0.379   
2         0.604                   ['Seweryn Goszczyński']         0.749   
3         0.995                      ['Francisco Canaro']         0.781   
4         0.990  ['Frédéric Chopin', 'Vladimir Horowitz']         0.210   

   duration_ms  energy  explicit                      id  instrumentalness  \
0       158648  0.1950         0  6KbQ3uYMLKb5jDxLF7wYDD             0.563   
1       282133  0.0135         0  6KuQTIu1KoTTkLXKrwlLPV             0.901   
2       104300  0.2200         0  6L63VW0PibdM1HDSBoqnoM             0.000   
3       180760  0.1300         0  6M94FkXd15sOAOQYRnWPN8             0.887   
4       687733  0.2040         0  6N6tiFZ9vLTSOIxkj8qKrd             0.908   

   key  liveness  loudness  mode  \
0   10    0.1510   -12.428     

In [5]:
print(data.columns)

Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo',
       'valence', 'year'],
      dtype='object')


In [48]:
# 1. Centroids
centroids = kmeans.cluster_centers_
print("Centroids:")
print(centroids)

# 2. Cluster Labels
cluster_labels = kmeans.labels_
print("Cluster Labels:")
print(cluster_labels)

# 3. Distribution of data points
cluster_distribution = pd.Series(cluster_labels).value_counts()
print("Distribution of data points across clusters:")
print(cluster_distribution)


Centroids:
[[0.4552245  0.26145836 0.33717671 0.69893279]
 [0.60943961 0.65167142 0.67757697 0.80463619]]
Cluster Labels:
[1 0 1 ... 0 0 0]
Distribution of data points across clusters:
1    77567
0    55373
Name: count, dtype: int64


In [16]:
df = pd.read_csv(r"H:\Sparsh Ehsaas Documents\Ehsaas\ML\Music Info.csv")
print(df.shape)
print(df.head())

(50683, 21)
             track_id             name           artist  \
0  TRIOREW128F424EAF0   Mr. Brightside      The Killers   
1  TRRIVDJ128F429B0E8       Wonderwall            Oasis   
2  TROUVHL128F426C441  Come as You Are          Nirvana   
3  TRUEIND128F93038C4      Take Me Out  Franz Ferdinand   
4  TRLNZBD128F935E4D8            Creep        Radiohead   

                                 spotify_preview_url              spotify_id  \
0  https://p.scdn.co/mp3-preview/4d26180e6961fd46...  09ZQ5TmUG8TSL56n0knqrj   
1  https://p.scdn.co/mp3-preview/d012e536916c927b...  06UfBBDISthj1ZJAtX4xjj   
2  https://p.scdn.co/mp3-preview/a1c11bb1cb231031...  0keNu0t0tqsWtExGM3nT1D   
3  https://p.scdn.co/mp3-preview/399c401370438be4...  0ancVQ9wEcHVd0RrGICTE4   
4  https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...  01QoK9DA7VTeTSE3MNzp4I   

                                                tags genre  year  duration_ms  \
0  rock, alternative, indie, alternative_rock, in...   NaN  2004       

In [18]:
print(df.columns)

Index(['track_id', 'name', 'artist', 'spotify_preview_url', 'spotify_id',
       'tags', 'genre', 'year', 'duration_ms', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')


In [41]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.preprocessing import image
import cv2
import numpy as np

data = pd.read_csv(r"H:\Sparsh Ehsaas Documents\Ehsaas\ML\Music Info.csv")

data.drop_duplicates(inplace=True, subset=['name'])

col_features = ['danceability', 'energy', 'valence', 'loudness', 
                'tempo', 'acousticness', 'liveness', 'speechiness', 'instrumentalness']

X = MinMaxScaler().fit_transform(data[col_features])

kmeans = KMeans(init="k-means++", n_clusters=10, random_state=15).fit(X) 
data['kmeans'] = kmeans.labels_

# Load emotion detection model
emotion_model_path = r"final_model_all_emotions.h5.keras"
emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["anger", "disgust", "fear","happiness", "sadness", "surprise", "neutral"]

# Confidence threshold for emotion classification
confidence_threshold = 0.5

# Function to process image and detect emotions
def process_image(image_path):
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Unable to read the image.")
        return []

    # Convert image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load the face detection model
    face_detection = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    if face_detection.empty():
        print("Error: Unable to load the face detection model.")
        return []

    # Detect faces in the image
    faces_detected = face_detection.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    emotions = []
    for (x, y, w, h) in faces_detected:
        roi_gray = gray_img[y:y+w, x:x+h]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255
        predictions = emotion_classifier.predict(img_pixels)

        # Apply confidence threshold
        max_confidence = np.max(predictions)
        if max_confidence >= confidence_threshold:
            max_index = np.argmax(predictions[0])
            predicted_emotion = EMOTIONS[max_index]
            emotions.append(predicted_emotion)

    return emotions

# Function to get music recommendations based on detected emotions
def get_results(emotions):
    NUM_RECOMMEND = 10
    recommendations = {}
    for emotion in emotions:
        if emotion == 'happiness':
            recommendations[emotion] = data[data['kmeans'] == 0]['name'].head(NUM_RECOMMEND)
        elif emotion == 'sadness':
            recommendations[emotion] = data[data['kmeans'] == 1]['name'].head(NUM_RECOMMEND)
        elif emotion == 'disgust':
            recommendations[emotion] = data[data['kmeans'] == 2]['name'].head(NUM_RECOMMEND)
        elif emotion == 'anger':
            recommendations[emotion] = data[data['kmeans'] == 3]['name'].head(NUM_RECOMMEND)
        elif emotion == 'fear':
            recommendations[emotion] = data[data['kmeans'] == 4]['name'].head(NUM_RECOMMEND)
        elif emotion == 'surprise':
            recommendations[emotion] = data[data['kmeans'] == 5]['name'].head(NUM_RECOMMEND)
        elif emotion == 'neutral':
            recommendations[emotion] = data[data['kmeans'] == 6]['name'].head(NUM_RECOMMEND)
        else:
            recommendations[emotion] = data[data['kmeans'] == 7]['name'].head(NUM_RECOMMEND)
    return recommendations


# Path to the image provided by the user
image_path = input("Enter the path to the image: ")

# Process the image and get detected emotions
emotions = process_image(image_path)

# Get music recommendations based on the detected emotions
recommendations = get_results(emotions)

# Print the detected emotions and corresponding song recommendations
print("Detected emotions:", emotions)
for emotion, songs in recommendations.items():
    print(f"Songs for {emotion}:")
    print(songs)

Enter the path to the image:  C:\Users\ehsaa\Downloads\anger.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
Detected emotions: ['anger']
Songs for anger:
4                    Creep
7             Karma Police
10        Under the Bridge
11       Seven Nation Army
14                 Fix You
18            Chasing Cars
22       Bohemian Rhapsody
27            No Surprises
43       Where Is My Mind?
63    Nothing Else Matters
Name: name, dtype: object
